In [66]:
from ortools.sat.python import cp_model
import itertools
import numpy as np
import pandas as pd

In [67]:
def init_employee(model, timeslots=4*11, days=5, employee="IDK"):
    """
    Set up one employee
    """
    # Create a list of all possible shifts
    employee_times = [[] for _ in range(days)]
    for i in range(days):
        for j in range(timeslots):
            employee_times[i].append(model.NewBoolVar('%s_e%i_%i' % (employee, i, j)))
    return employee_times

def create_no_gaps_constraint(model, emplyee_times):
    """
    Create a constraint that there are no gaps in the schedule
    """
    # First add a new variable that is 0 (false) if the employee works at both index i and i+1 and 1 if they are working at i and not at i+1

    DAYS = len(emplyee_times)
    TIMESLOTS = len(emplyee_times[0])

    # Create a list of all possible shifts
    emplyee_no_gaps_constraint = [[],]*DAYS
    for i in range(DAYS):
        for j in range(TIMESLOTS-1):
            emplyee_no_gaps_constraint[i].append(model.NewBoolVar('e%i_%i' % (i, j)))
            
            # Add a constraint that this variable is true if the employee is working at index i but not i+1
            model.Add(emplyee_times[i][j] > emplyee_times[i][j+1]).OnlyEnforceIf(emplyee_no_gaps_constraint[i][j])

    # Add a constraint that the sum of all these variables is at most 1
    for i in range(DAYS):
        model.Add(sum(emplyee_no_gaps_constraint[i]) <= 1)

def create_desired_times_constraint(model, employees_times, desired_times):
    """ 
    Create a constraint that at least desired_times[i,j] (integer) employees must work at timeslot j on day i 
    """
    N_EMPLOYEES = len(employees_times)
    DAYS = len(employees_times[0])
    TIMESLOTS = len(employees_times[0][0])

    for i in range(DAYS):
        for j in range(TIMESLOTS):
            model.Add(sum([employees_times[k][i][j] for k in range(N_EMPLOYEES)]) >= desired_times[i][j])


In [68]:
def init_schedule(DAY_START, DAY_END, dM):
    timeslots = {"{}:{}".format(h, m): i for i, (h, m) in enumerate(
    itertools.product(range(DAY_START, DAY_END), range(0, 60, dM)))}
    ind_to_timeslot = {i: t for t, i in timeslots.items()}
    schedule = pd.DataFrame(0, index=timeslots.keys(), columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
    return np.array(schedule), schedule, timeslots, ind_to_timeslot

In [69]:
employees = ["Tilda", "Annica", "Christina", "Kristina", "Liselott", "Lillian", "Malin R", "Ghada", "Nafiseh", "Laila", "Maria", "Lotta", "Cecilia", "Lina", "Anders"]
employees_ind = {i: e for i, e in enumerate(employees)}

In [70]:
desired_times = pd.read_csv("desired.csv", delimiter="\t")

DAYS = 5
DAY_START = 6
DAY_END = 18
dM = 15

timeslots = {i: "{}:{}".format(h, m) for i, (h, m) in enumerate(
             itertools.product(range(DAY_START, DAY_END), range(0, 60, dM)))}

del timeslots[47]
assert len(timeslots) == desired_times.shape[0]

In [71]:
# Create the model
model = cp_model.CpModel()

In [72]:
# For each employee, initialize their schedule
employees_times = {}
for i in employees_ind.keys():
    employees_times[i] = init_employee(model, timeslots=len(timeslots), days=DAYS, employee=employees_ind[i])

In [73]:
print(employees_times[0][0][0])

Tilda_e0_0


In [74]:
# For each employee, create a constraint that there are no gaps in their schedule
for i in employees_ind.keys():
    create_no_gaps_constraint(model, employees_times[i])

In [75]:
# Add a constraint that at least desired_times[i,j] (integer) employees must work at timeslot j on day i
create_desired_times_constraint(model, list(employees_times.values()), np.array(desired_times).T)

In [76]:
# Minimize the distance from 40 hours per week
# For each employee, create a variable that is the sum of all their shifts
employee_numshifts = []
for i in employees_ind.keys():
    employee_numshifts.append(model.NewIntVar(0, 300, '%s_e%i' % (employees_ind[i], i)))
    model.Add(employee_numshifts[i] == sum(employees_times[i][j//len(timeslots)][j%len(timeslots)] for j in range(len(timeslots)*DAYS)))

# For each employee, create a variable that is the distance from 40 hours
desired_shifts = 40*4 # 40 15 minute shifts per week

employee_hours_diff = []
for i in employees_ind.keys():
    employee_hours_diff.append(model.NewIntVar(-160, 160, '%s_e%i_employee_hours_diff' % (employees_ind[i], i)))
    diff_bool = model.NewBoolVar('%s_e%i_bool' % (employees_ind[i], i))

    model.Add(employee_numshifts[i] >= desired_shifts).OnlyEnforceIf(diff_bool)
    model.Add(employee_numshifts[i] < desired_shifts).OnlyEnforceIf(diff_bool.Not())

    model.Add(employee_hours_diff[i] == employee_numshifts[i] - desired_shifts).OnlyEnforceIf(diff_bool)
    model.Add(employee_hours_diff[i] == desired_shifts - employee_numshifts[i]).OnlyEnforceIf(diff_bool.Not())
    # model.Add(employee_hours_diff[i] >= 0)

# Minimize the sum of the distances from 40 hours
model.Minimize(sum(employee_hours_diff))

In [77]:
# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [78]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f'Minimum of objective function: {solver.ObjectiveValue()}\n')
    solution = {}
    for i in employees_ind.keys():
        solution[employees_ind[i]] = np.zeros((DAYS, len(timeslots)))
        for j in range(DAYS):
            for k in range(len(timeslots)):
                # print(solver.Value(employees_times[i][j][k]))
                solution[employees_ind[i]][j, k] = solver.Value(employees_times[i][j][k])

    # for i in employees_ind.keys():
    #     print(f'{employees_ind[i]}: {solver.Value(employee_numshifts[i])} minutes')
else:
    print('No solution found.')

# Statistics.
print('\nStatistics')
print(f'  status   : {solver.StatusName(status)}')
print(f'  conflicts: {solver.NumConflicts()}')
print(f'  branches : {solver.NumBranches()}')
print(f'  wall time: {solver.WallTime()} s')

Minimum of objective function: 10.0


Statistics
  status   : FEASIBLE
  conflicts: 3307205
  branches : 18460121
  wall time: 1693.8653896300002 s


In [79]:
# Solution of Anders to dataframe

# Sum over all employees
solution_df = pd.DataFrame(index=timeslots.keys(), columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], data=np.zeros(solution["Tilda"].T.shape))
for i in employees_ind.keys():
    solution_df += pd.DataFrame(index=timeslots.keys(), columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], data=solution[employees_ind[i]].T)




# solution_df = pd.DataFrame(index=timeslots.keys(), columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], data=solution["Tilda"].T)

In [80]:
for c in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
    print(sum(solution_df[c]))

478.0
480.0
482.0
484.0
486.0
